In [1]:
import time
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymoo.core.problem import ElementwiseProblem
from pymoo.optimize import minimize
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.core.sampling import Sampling

from pymoo.operators.crossover.pntx import (
    SinglePointCrossover,
    TwoPointCrossover,
)
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PolynomialMutation
from pymoo.operators.mutation.bitflip import BFM
from pymoo.operators.repair.rounding import RoundingRepair
from pymoo.core.mutation import Mutation
from pymoo.core.crossover import Crossover

import pop_ga
import revenue_estimation

from pop_ga import SwapCrossover, SwapMutation, SKUPopulationSampling

GEN_SIZE = 100
POP_SIZE = 10
CFG = {
    "sku_num": 36,  # no of sku
    "h": 8,  # week horizon
    "price_opt_num": 4,  # num of pricing options dvar
    "ndf": 2,  # num of display, feature dvar
    "lim_pro_per_cate_xu": 36,  # num of promotion items upper bound
    "lim_dis_per_cate_xu": 36,  # num of display items upper bound
    "lim_fea_per_cate_xu": 36,  # num of feature items upper bound
    "constraint_num": 2,  # simplify to 2 high level constraints
}

start_week = 1375
period = 8
zscore = pd.read_csv("../assets/Z_scores.csv")
selected_sku_list = zscore.iloc[:,0].tolist()


In [2]:
rev_est = revenue_estimation.RevenueEstimation()
problem = pop_ga.PromotionOptimizationProblem(
    cfg=CFG,
    rev_est=rev_est,
    selected_sku_list=selected_sku_list,
    start_week=start_week,
    period=period,
)

algo_best_ops = GA(
    pop_size=POP_SIZE,
    sampling=pop_ga.SKUPopulationSampling(cfg=problem.cfg, pop_size=POP_SIZE),
    mutation=PolynomialMutation(prob=0.6, eta=20),
    crossover=TwoPointCrossover(prob=0.4),
    eliminate_duplicates=True,
)
tested_combinations = set()

results = []
total_time = 0
start_time = time.time()

result = minimize(
    problem,
    algo_best_ops,
    ("n_gen", GEN_SIZE),
    seed=2,
    save_history=True,
    verbose=True,
)

elapsed_time = time.time() - start_time
total_time += elapsed_time

# Store results
results.append(
    {
        "result": result,
        "objective_value": result.F[0],
        "elapsed_time": elapsed_time,
    }
)


print(results)
sorted_results = sorted(results, key=lambda x: x["objective_value"])
with open(f"results/results_{start_week}.pkl", "wb") as f:
    pickle.dump(sorted_results, f)


# After all permutations, print the total time taken
print("Total time for all permutations (best):", total_time)


n_gen  |  n_eval  |     cv_min    |     cv_avg    |     f_avg     |     f_min    
     1 |       10 |  0.000000E+00 |  0.000000E+00 | -6.341757E+04 | -7.085694E+04
     2 |       20 |  0.000000E+00 |  0.000000E+00 | -6.848140E+04 | -7.302860E+04
     3 |       30 |  0.000000E+00 |  0.000000E+00 | -7.042657E+04 | -7.390903E+04
     4 |       40 |  0.000000E+00 |  0.000000E+00 | -7.239332E+04 | -7.390903E+04
     5 |       50 |  0.000000E+00 |  0.000000E+00 | -7.365107E+04 | -7.419460E+04
     6 |       60 |  0.000000E+00 |  0.000000E+00 | -7.406354E+04 | -7.541936E+04
     7 |       70 |  0.000000E+00 |  0.000000E+00 | -7.452311E+04 | -7.587103E+04
     8 |       80 |  0.000000E+00 |  0.000000E+00 | -7.529222E+04 | -7.587168E+04
     9 |       90 |  0.000000E+00 |  0.000000E+00 | -7.568545E+04 | -7.591903E+04
    10 |      100 |  0.000000E+00 |  0.000000E+00 | -7.587861E+04 | -7.592903E+04
    11 |      110 |  0.000000E+00 |  0.000000E+00 | -7.588119E+04 | -7.592903E+04
    12 |      12

In [3]:
def _to_discount_values(can_sol):
    """
    Converts a binary array to a discount values array with preserved columns.

    Transforms the first four binary columns of an input array into a single
    discount column based on predefined discount rates (0.8, 0.6, 0.4, 0.2).
    The display and feature columns from the input are preserved in the output.

    Parameters:
    - can_sol (numpy.ndarray): The input binary array with shape (n, 6).

    Returns:
    - pandas.DataFrame: The transformed array with shape (n, 3), including the
    discount value and the original last two columns.
    """
    discount_values = np.array([0.8, 0.6, 0.4, 0.2])
    result = np.zeros((can_sol.shape[0], 3))

    # discount_cols = can_sol[:, :4]

    for i, row in enumerate(can_sol):
        if row.sum() > 0:
            discount_index = np.argmax(row == 1)
            result[i, 0] = discount_values[discount_index]
        result[i, 1] = int(row[5])
        result[i, 2] = int(row[4])

    result_df = pd.DataFrame(result, columns=["Discount", "Feature", "Display"])

    return result_df

from copy import deepcopy


In [4]:
can_sol = deepcopy(result.X)
can_sol = can_sol.reshape(288, 6)
solution = _to_discount_values(can_sol)

sales_dir = '../assets/processed_sales.csv'

sales = pd.read_csv(sales_dir)
sku_list = sorted(sales['SKU'].unique())

start = start_week
end = start_week + period

idx_frame = [(SKU, Time_ID) for Time_ID in range(start, end) for SKU in sku_list]
idx_frame = pd.DataFrame(idx_frame, columns=["SKU", "Time_ID"])

output_df = pd.concat([idx_frame, solution], axis=1)

solution_id = "_".join([str(POP_SIZE), str(GEN_SIZE), f"{CFG['lim_pro_per_cate_xu']:02}", f"{CFG['lim_dis_per_cate_xu']:02}", f"{CFG['lim_fea_per_cate_xu']:02}", str(start_week), f"{period:02}"])

avg_weekly_feature = output_df.groupby(['Time_ID'])['Feature'].sum().mean()
avg_weekly_display = output_df.groupby(['Time_ID'])['Display'].sum().mean()

avg_weekly_discount = output_df[output_df['Discount'] > 0].groupby('Time_ID').size().mean()
avg_discount_values = output_df[output_df['Discount'] > 0].groupby('Time_ID')['Discount'].mean().mean()
objective_value = results[0]['objective_value']

print(f'Objective Value: {objective_value}')
print(f"Average Weekly Features: {avg_weekly_feature}")
print(f"Average Weekly Display: {avg_weekly_display}")
print(f"Average Weekly Discount: {avg_weekly_discount}")
print(f"Average Weekly Discount Amount: {avg_discount_values}")

output_df.to_csv(f'results/Output_{solution_id}.csv')   

Objective Value: -76364.42116389137
Average Weekly Features: 6.0
Average Weekly Display: 4.625
Average Weekly Discount: 11.5
Average Weekly Discount Amount: 0.487481684981685
